### 1. 라이브러리 불러오기 및 세션 생성

In [1]:
import json
import pandas as pd
from pyspark import SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType
from pyspark.sql.functions import explode, map_keys, col, first, get_json_object, array, to_json, struct, regexp_replace, split, length

In [2]:
# Spark 세션 생성
spark = SparkSession \
        .builder \
        .appName("processingJson") \
        .getOrCreate()

23/11/20 16:46:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 2. 데이터 불러오기

In [3]:
# 분할된 JSON 파일 경로 선택하는 함수
def nth_json_path(n):
    return f'/Users/b06/Desktop/yeardream/medi-05/data/naverplace_meta/naverplace_meta_{n}.json'

In [4]:
# 첫번째 JSON 파일 데이터 불러오기
n = 1
data = spark.read.json(nth_json_path(n))

23/11/20 16:46:32 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


### 3. 변수

In [5]:
columns = data.columns

In [6]:
hospital_bases = [c for c in columns if "HospitalBase" in c]

In [7]:
target_columns = [
    'id',
    'name', 
    'road', 
    'reviewSettings', 
    'conveniences', 
    'keywords', 
    'phone', 
    'virtualPhone', 
    'naverBookingUrl', 
    'talktalkUrl', 
    'paymentInfo', 
    'homepages',
    'visitorReviewsTotal',
    'description',
    'Images'
]

In [8]:
string_columns = [
    'id',
    'name', 
    'road',
    'phone',
    'virtualPhone',
    'naverBookingUrl',
    'talktalkUrl',
    'visitorReviewsTotal'
]

In [9]:
description_columns = [
    'id',
    'description'
]

In [10]:
struct_columns = [
    'id',
    'reviewSettings',
    'homepages'
]

In [11]:
review_keyword_columns = [
    'id',
    'reviewSettings.keyword'
]

In [12]:
homepages_columns = [
    'id',
    'homepages.repr.url',
    'homepages.repr.type',
    'homepages.repr.isDeadUrl',
    'homepages.repr.landingUrl'
]

In [13]:
conveniences_columns = [
    'id',
    'conveniences'
]

In [14]:
keywords_columns = [
    'id',
    'keywords'
]

In [15]:
payments_columns = [
    'id',
    'paymentInfo'
]

In [16]:
string_columns_schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("road", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("virtualPhone", StringType(), True),
    StructField("naverBookingUrl", StringType(), True),
    StructField("talktalkUrl", StringType(), True),
    StructField("visitorReviewsTotal", IntegerType(), True)
])

In [17]:
description_columns_schema = StructType([
    StructField("id", StringType(), True),
    StructField("description", StringType(), True),
    StructField("description_length", IntegerType(), True)
])

In [18]:
review_keyword_columns_schema = StructType([
    StructField("id", StringType(), True),
    StructField("review_keyword", StringType(), True)
])

In [19]:
homepages_columns_schema = StructType([
    StructField("id", StringType(), True),
    StructField("url", StringType(), True),
    StructField("type", StringType(), True),
    StructField("isDeadUrl", BooleanType(), True),
    StructField("landingUrl", StringType(), True)
])

In [20]:
conveniences_columns_schema = StructType([
    StructField("id", StringType(), True),
    StructField("conveniences", StringType(), True),
])

In [21]:
keywords_columns_schema = StructType([
    StructField("id", StringType(), True),
    StructField("keywords", StringType(), True),
])

In [22]:
payments_columns_schema = StructType([
    StructField("id", StringType(), True),
    StructField("payments", StringType(), True),
])

In [23]:
string_table = spark.createDataFrame([], string_columns_schema)

In [24]:
description_table = spark.createDataFrame([], description_columns_schema)

In [25]:
review_keyword_table = spark.createDataFrame([], review_keyword_columns_schema)

In [26]:
homepages_table = spark.createDataFrame([], homepages_columns_schema)

In [27]:
conveniences_table = spark.createDataFrame([], conveniences_columns_schema)

In [28]:
keywords_table = spark.createDataFrame([], keywords_columns_schema)

In [29]:
payments_table = spark.createDataFrame([], payments_columns_schema)

### 4. 함수

In [30]:
def get_table(df, columns, table):
    get_columns = df.select(columns)
    row = remove_null(get_columns)
    return table.union(row)

In [31]:
def remove_null(df):
    return df.filter(~col('name').isNull())

In [37]:
def get_description_table(df, columns, table):
    cols = df.select(columns)
    rows = remove_null(cols)
    rows = rows.withColumn('description', regexp_replace('description', '\n', ' '))
    rows = rows.withColumn('description_length', length('description'))
    return table.union(rows)

In [32]:
def preprocessing_review_keyword(review_keyword_row):
    review_keyword_row = review_keyword_row.withColumnRenamed("keyword", "review_keyword")
    review_keyword_row = review_keyword_row.withColumn("review_keyword", regexp_replace("review_keyword", " & ", ", "))
    review_keyword_row = review_keyword_row.withColumn("review_keyword", regexp_replace("review_keyword", "[()]", ""))
    review_keyword_row = review_keyword_row.withColumn("review_keyword", explode(split(col("review_keyword"), ", ")))
    return review_keyword_row

In [33]:
def get_review_keyword_table(struct_df, review_keyword_columns, review_keyword_df):
    get_review_keyword_columns = struct_df.select(review_keyword_columns)
    review_keyword_row = remove_null(get_review_keyword_columns)
    review_keyword_row = preprocessing_review_keyword(review_keyword_row)
    return review_keyword_df.union(review_keyword_row)

In [34]:
def check_null(df, column):
    cnt = df.filter(col(column).isNull()).count()
    return True if cnt == 10 else False

In [35]:
def get_homepages_table(struct_df, homepages_columns, homepages_table):
    if check_null(struct_df, 'homepages.repr'):
        return homepages_table
    else:
        return get_table(struct_df, homepages_columns, homepages_table)

In [36]:
def get_table_and_explode(df, columns, table, column):
    if check_null(df, column):
        return table
    else:
        get_columns = df.select(columns)
        row = remove_null(get_columns)
        rows = row.withColumn(column, explode(row[column]))
        return table.union(rows)

### 5. 데이터 전처리

In [38]:
for hospital_base in hospital_bases:    
    hospital_base_data = data.select(hospital_base)

    get_columns = [col(hospital_base + "." + t).alias(t) for t in target_columns]
    df = hospital_base_data.select(get_columns)
    
    string_table = get_table(df, string_columns, string_table)
    description_table = get_description_table(df, description_columns, description_table)
    struct_df = df.select(struct_columns)
    review_keyword_table = get_review_keyword_table(struct_df, review_keyword_columns, review_keyword_table)
    homepages_table = get_homepages_table(struct_df, homepages_columns, homepages_table)
    conveniences_table = get_table_and_explode(df, conveniences_columns, conveniences_table, 'conveniences')
    keywords_table = get_table_and_explode(df, keywords_columns, keywords_table, 'keywords')
    payments_table = get_table_and_explode(df, payments_columns, payments_table, 'paymentInfo')

AnalysisException: [NUM_COLUMNS_MISMATCH] UNION can only be performed on inputs with the same number of columns, but the first input has 3 columns and the second input has 2 columns.;
'Union false, false
:- LogicalRDD [id#530, description#531, description_length#532], false
+- Project [description#627, length(description#627) AS description_length#629]
   +- Project [regexp_replace(description#577, 
,  , 1) AS description#627]
      +- Project [description#577]
         +- Filter NOT isnull(name#565)
            +- Project [description#577, name#565]
               +- Project [HospitalBase:11779766#79.id AS id#564, HospitalBase:11779766#79.name AS name#565, HospitalBase:11779766#79.road AS road#566, HospitalBase:11779766#79.reviewSettings AS reviewSettings#567, HospitalBase:11779766#79.conveniences AS conveniences#568, HospitalBase:11779766#79.keywords AS keywords#569, HospitalBase:11779766#79.phone AS phone#570, HospitalBase:11779766#79.virtualPhone AS virtualPhone#571, HospitalBase:1

In [ ]:
# string_table.show(50)
# review_keyword_table.show(50)
# homepages_table.show(50)
# conveniences_table.show(50)
# keywords_table.show(50)
payments_table.show(50)

### Test codes
---

In [ ]:
# rows = rows.collect()
# for row in rows:
#     print(row)

In [ ]:
hb = hospital_bases[3]
hb_data = data.select(hb)
get_columns = [col(hb + "." + t).alias(t) for t in target_columns]
df = hb_data.select(get_columns)

In [ ]:
print(hospital_bases[1])

In [ ]:
cols = df.select('description')
rows = remove_null(cols)
rows = rows.withColumn('description', regexp_replace('description', '\n', ' '))
rows = rows.withColumn('description_length', length('description'))
rows.show()

In [ ]:
from pyspark.sql.functions import regexp_replace, length

# description 컬럼에서 '\n'을 제거
df = df.withColumn('description', regexp_replace('description', '\n', ' '))

# 수정된 description의 길이를 새로운 컬럼 description_length에 저장
df = df.withColumn('description_length', length('description'))

# 결과 확인
df.show()